In [ ]:
# This tutorial is available for download as a Jupyter notebook


# CaliBrain Library Overview: Core Components Tutorial

This tutorial provides a comprehensive introduction to the main components 
of the CaliBrain library for EEG/MEG source localization and uncertainty quantification.

CaliBrain is designed to help researchers:
1. Simulate realistic brain source activity and sensor measurements
2. Apply source estimation algorithms (Gamma-MAP, eLORETA, etc.)
3. Quantify uncertainty in source estimates
4. Evaluate calibration quality of uncertainty estimates
5. Visualize results and generate comprehensive reports

Author: CaliBrain Development Team
License: MIT


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Import CaliBrain components
from calibrain import (
    Benchmark,
    SourceSimulator, 
    SensorSimulator,
    LeadfieldBuilder,
    SourceEstimator,
    UncertaintyEstimator,
    MetricEvaluator,
    Visualizer
)

#==============================================================================
# 1. LEADFIELD BUILDER - Foundation of Forward Modeling
#==============================================================================

def tutorial_leadfield_builder():
    """
    LeadfieldBuilder: Creates the forward model connecting brain sources to sensors
    
    The leadfield matrix L maps source activity x to sensor measurements y:
    y = L @ x + noise
    
    Key Features:
    - Support for both MEG and EEG modalities
    - Fixed and free source orientations
    - Integration with MNE-Python for realistic head models
    - Efficient matrix storage and loading
    """
    print("=" * 60)
    print("1. LEADFIELD BUILDER TUTORIAL")
    print("=" * 60)
    
    # Configuration for leadfield simulation
    leadfield_config = {
        "subject": "fsaverage",           # Standard brain template
        "subjects_dir": "./data/subjects", # MNE subjects directory
        "spacing": "ico4",                # Source space resolution
        "modality": "eeg",                # EEG or MEG
        "orientation": "fixed",           # "fixed" or "free"
        "montage": "standard_1020",       # EEG electrode layout
        "info_sfreq": 250,               # Sampling frequency
    }
    
    # Initialize leadfield builder
    leadfield_builder = LeadfieldBuilder(config=leadfield_config)
    
    print(f"Building leadfield for {leadfield_config['modality'].upper()} with {leadfield_config['orientation']} orientations...")
    
    # Build or load leadfield matrix
    leadfield_matrix = leadfield_builder.simulate()
    
    print(f"Leadfield shape: {leadfield_matrix.shape}")
    print(f"  - Sensors: {leadfield_matrix.shape[0]}")
    print(f"  - Sources: {leadfield_matrix.shape[1]}")
    print(f"  - Condition number: {np.linalg.cond(leadfield_matrix):.2e}")
    
    return leadfield_matrix

#==============================================================================
# 2. SOURCE SIMULATOR - Brain Activity Generation
#==============================================================================

def tutorial_source_simulator():
    """
    SourceSimulator: Generates realistic brain source activity patterns
    
    Creates synthetic dipole activity with:
    - Event-related potential (ERP) waveforms
    - Configurable amplitude, timing, and frequency content
    - Support for multiple active sources
    - Realistic temporal dynamics
    """
    print("\n" + "=" * 60)
    print("2. SOURCE SIMULATOR TUTORIAL")
    print("=" * 60)
    
    # ERP configuration - defines the temporal profile
    ERP_config = {
        "tmin": -0.5,           # Start time (seconds)
        "tmax": 0.5,            # End time (seconds)
        "stim_onset": 0.0,      # Stimulus onset time
        "sfreq": 250,           # Sampling frequency (Hz)
        "fmin": 1,              # Low-pass filter cutoff
        "fmax": 5,              # High-pass filter cutoff
        "amplitude": 50.0,      # Peak amplitude (nAm)
        "random_erp_timing": True,  # Randomize exact timing
    }
    
    # Initialize source simulator
    source_simulator = SourceSimulator(ERP_config=ERP_config)
    
    # Simulation parameters
    n_sources = 10000        # Total sources in source space
    n_active = 5            # Number of active sources
    n_trials = 50           # Number of trials to simulate
    
    print(f"Simulating {n_active} active sources out of {n_sources} total sources...")
    print(f"Generating {n_trials} trials with ERP amplitude {ERP_config['amplitude']} nAm")
    
    # Generate source activity
    source_data, active_indices = source_simulator.simulate(
        n_sources=n_sources,
        n_active=n_active,
        n_trials=n_trials
    )
    
    print(f"Source data shape: {source_data.shape}")
    print(f"Active source indices: {active_indices}")
    print(f"Peak amplitude: {np.max(np.abs(source_data)):.2f} nAm")
    
    # Visualize a sample ERP waveform
    time_vector = np.linspace(ERP_config['tmin'], ERP_config['tmax'], source_data.shape[1])
    active_source_activity = source_data[active_indices[0], :, 0]  # First active source, first trial
    
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(time_vector, active_source_activity)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (nAm)')
    plt.title('Simulated ERP Waveform')
    plt.axvline(0, color='r', linestyle='--', alpha=0.7, label='Stimulus onset')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.hist(np.max(np.abs(source_data), axis=1).flatten(), bins=50, alpha=0.7)
    plt.xlabel('Peak Amplitude (nAm)')
    plt.ylabel('Count')
    plt.title('Distribution of Source Amplitudes')
    plt.axvline(ERP_config['amplitude'], color='r', linestyle='--', label='Target amplitude')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    return source_data, active_indices, time_vector

#==============================================================================
# 3. SENSOR SIMULATOR - Forward Modeling with Noise
#==============================================================================

def tutorial_sensor_simulator(leadfield_matrix, source_data):
    """
    SensorSimulator: Converts source activity to sensor measurements
    
    Applies the forward model: y = L @ x + noise
    - Adds realistic sensor noise
    - Supports different noise types (white, colored, realistic)
    - Configurable signal-to-noise ratios
    """
    print("\n" + "=" * 60)
    print("3. SENSOR SIMULATOR TUTORIAL")
    print("=" * 60)
    
    # Initialize sensor simulator
    sensor_simulator = SensorSimulator()
    
    # Noise configuration
    noise_levels = [0.0, 0.3, 0.7, 0.9]  # Different SNR levels
    
    print("Simulating sensor measurements with different noise levels...")
    
    sensor_data_clean = None
    sensor_data_noisy = {}
    
    for alpha_snr in noise_levels:
        print(f"  - Alpha SNR: {alpha_snr:.1f}")
        
        # Generate sensor measurements
        y_clean, y_noisy = sensor_simulator.simulate(
            source_data=source_data,
            leadfield=leadfield_matrix,
            alpha_SNR=alpha_snr,
            noise_type="white"
        )
        
        if sensor_data_clean is None:
            sensor_data_clean = y_clean
            
        sensor_data_noisy[alpha_snr] = y_noisy
        
        # Calculate actual SNR
        signal_power = np.var(y_clean)
        noise_power = np.var(y_noisy - y_clean)
        actual_snr_db = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else np.inf
        
        print(f"    Actual SNR: {actual_snr_db:.1f} dB")
    
    print(f"Sensor data shape: {sensor_data_clean.shape}")
    print(f"  - Channels: {sensor_data_clean.shape[0]}")
    print(f"  - Time points: {sensor_data_clean.shape[1]}")
    print(f"  - Trials: {sensor_data_clean.shape[2]}")
    
    # Visualize sensor data
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    time_vector = np.linspace(-0.5, 0.5, sensor_data_clean.shape[1])
    
    for i, alpha_snr in enumerate([0.0, 0.3, 0.7, 0.9]):
        ax = axes[i//2, i%2]
        
        # Plot first few channels
        for ch in range(min(5, sensor_data_noisy[alpha_snr].shape[0])):
            ax.plot(time_vector, sensor_data_noisy[alpha_snr][ch, :, 0], alpha=0.7)
        
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Amplitude (V)')
        ax.set_title(f'Sensor Data (α_SNR = {alpha_snr:.1f})')
        ax.axvline(0, color='r', linestyle='--', alpha=0.5)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return sensor_data_clean, sensor_data_noisy

#==============================================================================
# 4. SOURCE ESTIMATOR - Inverse Problem Solving
#==============================================================================

def tutorial_source_estimator(sensor_data, leadfield_matrix):
    """
    SourceEstimator: Solves the inverse problem to estimate brain sources
    
    Implements multiple algorithms:
    - Gamma-MAP: Hierarchical Bayesian approach with automatic relevance determination
    - eLORETA: Exact low-resolution electromagnetic tomography
    - sLORETA: Standardized LORETA
    - dSPM: Dynamic statistical parametric mapping
    """
    print("\n" + "=" * 60)
    print("4. SOURCE ESTIMATOR TUTORIAL")
    print("=" * 60)
    
    # Initialize source estimator
    source_estimator = SourceEstimator()
    
    # Test different algorithms
    algorithms = ['gamma_map', 'eloreta', 'sloreta', 'dspm']
    
    source_estimates = {}
    
    for algorithm in algorithms:
        print(f"\nTesting {algorithm.upper()} algorithm...")
        
        # Algorithm-specific parameters
        if algorithm == 'gamma_map':
            params = {
                'alpha': 0.01,           # Regularization parameter
                'beta': 1.0,             # Noise precision
                'max_iter': 100,         # Maximum iterations
                'tol': 1e-4             # Convergence tolerance
            }
        else:
            params = {
                'alpha': 0.01           # Regularization parameter
            }
        
        try:
            # Estimate sources
            estimated_sources = source_estimator.estimate(
                sensor_data=sensor_data[:, :, 0],  # Use first trial
                leadfield=leadfield_matrix,
                method=algorithm,
                **params
            )
            
            source_estimates[algorithm] = estimated_sources
            
            print(f"  Estimated sources shape: {estimated_sources.shape}")
            print(f"  Peak amplitude: {np.max(np.abs(estimated_sources)):.2e}")
            print(f"  Sparsity (% zero): {100 * np.mean(np.abs(estimated_sources) < 1e-10):.1f}%")
            
        except Exception as e:
            print(f"  Error with {algorithm}: {str(e)}")
            source_estimates[algorithm] = None
    
    # Visualize source estimates
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    for i, algorithm in enumerate(algorithms):
        ax = axes[i//2, i%2]
        
        if source_estimates[algorithm] is not None:
            # Plot source amplitude over time
            max_amplitude = np.max(np.abs(source_estimates[algorithm]), axis=1)
            ax.stem(range(len(max_amplitude)), max_amplitude, basefmt=' ')
            ax.set_xlabel('Source Index')
            ax.set_ylabel('Max Amplitude')
            ax.set_title(f'{algorithm.upper()} Source Estimates')
            ax.set_yscale('log')
        else:
            ax.text(0.5, 0.5, f'{algorithm.upper()}\nNot Available', 
                   ha='center', va='center', transform=ax.transAxes)
        
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return source_estimates

#==============================================================================
# 5. UNCERTAINTY ESTIMATOR - Confidence Interval Computation
#==============================================================================

def tutorial_uncertainty_estimator(sensor_data, leadfield_matrix, source_estimates):
    """
    UncertaintyEstimator: Quantifies uncertainty in source estimates
    
    Computes confidence intervals using:
    - Posterior covariance from Bayesian methods
    - Bootstrap resampling
    - Cross-validation estimates
    - Theoretical approximations
    """
    print("\n" + "=" * 60)
    print("5. UNCERTAINTY ESTIMATOR TUTORIAL")
    print("=" * 60)
    
    # Initialize uncertainty estimator
    uncertainty_estimator = UncertaintyEstimator()
    
    # Choose method with available source estimates
    available_method = None
    for method in ['gamma_map', 'eloreta', 'sloreta']:
        if source_estimates.get(method) is not None:
            available_method = method
            break
    
    if available_method is None:
        print("No source estimates available for uncertainty quantification")
        return None
    
    print(f"Computing uncertainty for {available_method.upper()} estimates...")
    
    # Configuration for uncertainty estimation
    confidence_levels = [0.68, 0.95, 0.99]  # 1σ, 2σ, 3σ confidence intervals
    
    uncertainty_results = {}
    
    for conf_level in confidence_levels:
        print(f"\nComputing {conf_level:.0%} confidence intervals...")
        
        try:
            # Estimate uncertainty
            lower_bounds, upper_bounds, point_estimates = uncertainty_estimator.estimate(
                sensor_data=sensor_data[:, :, :10],  # Use first 10 trials
                leadfield=leadfield_matrix,
                method=available_method,
                confidence_level=conf_level,
                n_bootstrap=100  # Number of bootstrap samples
            )
            
            uncertainty_results[conf_level] = {
                'lower': lower_bounds,
                'upper': upper_bounds,
                'point': point_estimates
            }
            
            # Calculate interval widths
            interval_widths = upper_bounds - lower_bounds
            
            print(f"  Confidence intervals computed successfully")
            print(f"  Mean interval width: {np.mean(interval_widths):.2e}")
            print(f"  Median interval width: {np.median(interval_widths):.2e}")
            
        except Exception as e:
            print(f"  Error computing {conf_level:.0%} CI: {str(e)}")
            uncertainty_results[conf_level] = None
    
    # Visualize uncertainty estimates
    if len(uncertainty_results) > 0:
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        for i, conf_level in enumerate(confidence_levels):
            if uncertainty_results[conf_level] is not None:
                result = uncertainty_results[conf_level]
                
                # Plot confidence intervals for first time point
                source_indices = range(min(100, len(result['point'])))
                
                axes[i].fill_between(source_indices, 
                                   result['lower'][:len(source_indices), 0],
                                   result['upper'][:len(source_indices), 0],
                                   alpha=0.3, label=f'{conf_level:.0%} CI')
                axes[i].plot(source_indices, result['point'][:len(source_indices), 0], 
                           'k-', label='Point estimate')
                
                axes[i].set_xlabel('Source Index')
                axes[i].set_ylabel('Amplitude')
                axes[i].set_title(f'{conf_level:.0%} Confidence Intervals')
                axes[i].legend()
                axes[i].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    return uncertainty_results

#==============================================================================
# 6. METRIC EVALUATOR - Performance Assessment
#==============================================================================

def tutorial_metric_evaluator(true_sources, estimated_sources, uncertainty_results, active_indices):
    """
    MetricEvaluator: Evaluates the quality of source estimates and uncertainty quantification
    
    Computes metrics including:
    - Localization error (distance between true and estimated sources)
    - False positive/negative rates
    - Calibration scores (reliability of confidence intervals)
    - Coverage probabilities
    - Sharpness of uncertainty estimates
    """
    print("\n" + "=" * 60)
    print("6. METRIC EVALUATOR TUTORIAL")
    print("=" * 60)
    
    # Initialize metric evaluator
    metric_evaluator = MetricEvaluator()
    
    # Find available estimates
    available_method = None
    for method in estimated_sources:
        if estimated_sources[method] is not None:
            available_method = method
            break
    
    if available_method is None:
        print("No source estimates available for evaluation")
        return None
    
    print(f"Evaluating {available_method.upper()} estimates...")
    
    # Prepare ground truth
    true_source_locations = active_indices
    estimated_data = estimated_sources[available_method]
    
    try:
        # Compute localization metrics
        localization_metrics = metric_evaluator.compute_localization_metrics(
            true_sources=true_sources,
            estimated_sources=estimated_data,
            true_active_indices=true_source_locations
        )
        
        print("\nLocalization Metrics:")
        for metric, value in localization_metrics.items():
            print(f"  {metric}: {value:.4f}")
        
        # Compute calibration metrics if uncertainty is available
        if uncertainty_results and len(uncertainty_results) > 0:
            print("\nCalibration Metrics:")
            
            for conf_level in uncertainty_results:
                if uncertainty_results[conf_level] is not None:
                    result = uncertainty_results[conf_level]
                    
                    calibration_metrics = metric_evaluator.compute_calibration_metrics(
                        true_sources=true_sources,
                        lower_bounds=result['lower'],
                        upper_bounds=result['upper'],
                        confidence_level=conf_level
                    )
                    
                    print(f"\n  {conf_level:.0%} Confidence Intervals:")
                    for metric, value in calibration_metrics.items():
                        print(f"    {metric}: {value:.4f}")
        
        # Visualization of evaluation results
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # Plot 1: True vs Estimated amplitude correlation
        true_amplitudes = np.max(np.abs(true_sources), axis=1)
        est_amplitudes = np.max(np.abs(estimated_data), axis=1)
        
        axes[0].scatter(true_amplitudes, est_amplitudes, alpha=0.6)
        axes[0].plot([0, np.max(true_amplitudes)], [0, np.max(true_amplitudes)], 'r--', alpha=0.7)
        axes[0].set_xlabel('True Amplitude')
        axes[0].set_ylabel('Estimated Amplitude')
        axes[0].set_title('True vs Estimated Amplitudes')
        axes[0].grid(True, alpha=0.3)
        
        # Plot 2: ROC-like curve for source detection
        thresholds = np.logspace(-6, -1, 50)
        tpr_list, fpr_list = [], []
        
        for threshold in thresholds:
            detected_sources = np.where(np.max(np.abs(estimated_data), axis=1) > threshold)[0]
            
            tp = len(np.intersect1d(detected_sources, active_indices))
            fp = len(detected_sources) - tp
            fn = len(active_indices) - tp
            tn = len(estimated_data) - len(active_indices) - fp
            
            tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
            fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
            
            tpr_list.append(tpr)
            fpr_list.append(fpr)
        
        axes[1].plot(fpr_list, tpr_list, 'b-', linewidth=2)
        axes[1].plot([0, 1], [0, 1], 'r--', alpha=0.7)
        axes[1].set_xlabel('False Positive Rate')
        axes[1].set_ylabel('True Positive Rate')
        axes[1].set_title('Source Detection Performance')
        axes[1].grid(True, alpha=0.3)
        
        # Plot 3: Calibration plot if uncertainty available
        if uncertainty_results and 0.95 in uncertainty_results and uncertainty_results[0.95] is not None:
            result = uncertainty_results[0.95]
            
            # Compute empirical coverage for different regions
            n_bins = 10
            coverage_empirical = []
            coverage_nominal = []
            
            for i in range(n_bins):
                start_idx = i * len(true_sources) // n_bins
                end_idx = (i + 1) * len(true_sources) // n_bins
                
                true_slice = true_sources[start_idx:end_idx]
                lower_slice = result['lower'][start_idx:end_idx]
                upper_slice = result['upper'][start_idx:end_idx]
                
                # Check coverage
                coverage = np.mean((true_slice >= lower_slice) & (true_slice <= upper_slice))
                coverage_empirical.append(coverage)
                coverage_nominal.append(0.95)  # Nominal 95% coverage
            
            axes[2].plot(coverage_nominal, coverage_empirical, 'bo-')
            axes[2].plot([0, 1], [0, 1], 'r--', alpha=0.7)
            axes[2].set_xlabel('Nominal Coverage')
            axes[2].set_ylabel('Empirical Coverage')
            axes[2].set_title('Calibration Plot')
            axes[2].grid(True, alpha=0.3)
        else:
            axes[2].text(0.5, 0.5, 'Uncertainty\nNot Available', 
                        ha='center', va='center', transform=axes[2].transAxes)
        
        plt.tight_layout()
        plt.show()
        
        return localization_metrics
        
    except Exception as e:
        print(f"Error in metric evaluation: {str(e)}")
        return None

#==============================================================================
# 7. VISUALIZER - Comprehensive Plotting and Analysis
#==============================================================================

def tutorial_visualizer(source_data, sensor_data, estimated_sources, active_indices, time_vector):
    """
    Visualizer: Creates comprehensive plots and visualizations
    
    Provides visualization for:
    - Source activity patterns over time
    - Sensor measurement topographies
    - Source estimate comparisons
    - Uncertainty quantification plots
    - Calibration analysis
    - Interactive brain plots (when MNE is available)
    """
    print("\n" + "=" * 60)
    print("7. VISUALIZER TUTORIAL")
    print("=" * 60)
    
    # Initialize visualizer
    visualizer = Visualizer(base_save_path="./tutorial_figures")
    
    print("Creating comprehensive visualizations...")
    
    try:
        # Create a comprehensive figure
        fig = plt.figure(figsize=(16, 12))
        
        # 1. Source activity over time
        ax1 = plt.subplot(3, 4, 1)
        for i, idx in enumerate(active_indices[:3]):  # First 3 active sources
            plt.plot(time_vector, source_data[idx, :, 0], label=f'Source {idx}')
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude (nAm)')
        plt.title('True Source Activity')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.axvline(0, color='r', linestyle='--', alpha=0.5)
        
        # 2. Sensor topography at peak time
        ax2 = plt.subplot(3, 4, 2)
        peak_time_idx = np.argmax(np.abs(sensor_data[:, :, 0]).sum(axis=0))
        sensor_values = sensor_data[:, peak_time_idx, 0]
        
        # Simple topography plot (circular arrangement)
        angles = np.linspace(0, 2*np.pi, len(sensor_values), endpoint=False)
        x = np.cos(angles)
        y = np.sin(angles)
        scatter = plt.scatter(x, y, c=sensor_values, cmap='RdBu_r', s=100)
        plt.colorbar(scatter, ax=ax2, shrink=0.8)
        plt.title('Sensor Topography')
        plt.axis('equal')
        plt.axis('off')
        
        # 3. Source estimate comparison
        ax3 = plt.subplot(3, 4, 3)
        available_method = None
        for method in estimated_sources:
            if estimated_sources[method] is not None:
                available_method = method
                break
        
        if available_method:
            true_amplitudes = np.max(np.abs(source_data), axis=(1, 2))
            est_amplitudes = np.max(np.abs(estimated_sources[available_method]), axis=1)
            
            plt.scatter(true_amplitudes, est_amplitudes, alpha=0.6)
            max_amp = max(np.max(true_amplitudes), np.max(est_amplitudes))
            plt.plot([0, max_amp], [0, max_amp], 'r--', alpha=0.7)
            plt.xlabel('True Amplitude')
            plt.ylabel('Estimated Amplitude')
            plt.title(f'{available_method.upper()} Estimates')
            plt.grid(True, alpha=0.3)
        
        # 4. Active source detection
        ax4 = plt.subplot(3, 4, 4)
        if available_method:
            source_indices = np.arange(len(est_amplitudes))
            colors = ['red' if idx in active_indices else 'blue' for idx in source_indices]
            plt.scatter(source_indices[:200], est_amplitudes[:200], c=colors[:200], alpha=0.6)
            plt.xlabel('Source Index')
            plt.ylabel('Estimated Amplitude')
            plt.title('Source Detection (Red=True Active)')
            plt.yscale('log')
            plt.grid(True, alpha=0.3)
        
        # 5-8. Time course plots for different noise levels
        if isinstance(sensor_data, dict):  # Multiple noise levels
            for i, (noise_level, data) in enumerate(list(sensor_data.items())[:4]):
                ax = plt.subplot(3, 4, 5 + i)
                
                # Plot first few channels
                for ch in range(min(3, data.shape[0])):
                    plt.plot(time_vector, data[ch, :, 0], alpha=0.7, label=f'Ch {ch+1}')
                
                plt.xlabel('Time (s)')
                plt.ylabel('Amplitude (V)')
                plt.title(f'Sensor Data (α={noise_level:.1f})')
                plt.grid(True, alpha=0.3)
                plt.axvline(0, color='r', linestyle='--', alpha=0.5)
                if i == 0:
                    plt.legend()
        
        # 9. Source power spectrum
        ax9 = plt.subplot(3, 4, 9)
        if len(active_indices) > 0:
            source_signal = source_data[active_indices[0], :, 0]
            freqs = np.fft.fftfreq(len(source_signal), 1/250)
            fft_signal = np.abs(np.fft.fft(source_signal))
            
            # Plot positive frequencies only
            pos_freqs = freqs[:len(freqs)//2]
            pos_fft = fft_signal[:len(freqs)//2]
            
            plt.semilogy(pos_freqs, pos_fft)
            plt.xlabel('Frequency (Hz)')
            plt.ylabel('Power')
            plt.title('Source Power Spectrum')
            plt.grid(True, alpha=0.3)
            plt.xlim(0, 25)
        
        # 10. Noise analysis
        ax10 = plt.subplot(3, 4, 10)
        if isinstance(sensor_data, dict):
            noise_levels = list(sensor_data.keys())
            snr_values = []
            
            for noise_level in noise_levels:
                if 'clean' in locals():
                    signal_power = np.var(sensor_data_clean)
                    total_power = np.var(sensor_data[noise_level])
                    snr_db = 10 * np.log10(signal_power / (total_power - signal_power + 1e-10))
                    snr_values.append(snr_db)
            
            if snr_values:
                plt.plot(noise_levels, snr_values, 'bo-')
                plt.xlabel('Alpha SNR')
                plt.ylabel('Actual SNR (dB)')
                plt.title('SNR vs Noise Level')
                plt.grid(True, alpha=0.3)
        
        # 11. Estimation error over time
        ax11 = plt.subplot(3, 4, 11)
        if available_method and len(active_indices) > 0:
            true_signal = source_data[active_indices[0], :, 0]
            est_signal = estimated_sources[available_method][active_indices[0], :]
            
            error = np.abs(true_signal - est_signal)
            plt.plot(time_vector, error)
            plt.xlabel('Time (s)')
            plt.ylabel('Absolute Error')
            plt.title('Estimation Error Over Time')
            plt.grid(True, alpha=0.3)
            plt.axvline(0, color='r', linestyle='--', alpha=0.5)
        
        # 12. Summary statistics
        ax12 = plt.subplot(3, 4, 12)
        stats_text = f"""
        SIMULATION SUMMARY
        
        Total Sources: {source_data.shape[0]}
        Active Sources: {len(active_indices)}
        Time Points: {source_data.shape[1]}
        Trials: {source_data.shape[2]}
        
        Peak True Amplitude: 
        {np.max(np.abs(source_data)):.2e} nAm
        
        """
        
        if available_method:
            stats_text += f"""
        Estimation Method: {available_method.upper()}
        Peak Est. Amplitude:
        {np.max(np.abs(estimated_sources[available_method])):.2e} nAm
        """
        
        plt.text(0.1, 0.9, stats_text, transform=ax12.transAxes, 
                fontfamily='monospace', fontsize=10, verticalalignment='top')
        plt.axis('off')
        
        plt.suptitle('CaliBrain Library Tutorial: Comprehensive Analysis', fontsize=16, y=0.95)
        plt.tight_layout()
        plt.show()
        
        print("Visualizations created successfully!")
        print(f"Figures would be saved to: {visualizer.base_save_path}")
        
    except Exception as e:
        print(f"Error in visualization: {str(e)}")

#==============================================================================
# 8. BENCHMARK - Automated Experimental Pipeline
#==============================================================================

def tutorial_benchmark():
    """
    Benchmark: Automated pipeline for comprehensive evaluation
    
    The Benchmark class orchestrates the entire workflow:
    - Configures simulation parameters
    - Runs experiments across parameter grids
    - Collects and organizes results
    - Generates comprehensive reports
    - Supports parallel execution for large-scale studies
    """
    print("\n" + "=" * 60)
    print("8. BENCHMARK TUTORIAL")
    print("=" * 60)
    
    # Configure the benchmark experiment
    ERP_config = {
        "tmin": -0.5,
        "tmax": 0.5,
        "sfreq": 250,
        "amplitude": 30.0,
    }
    
    # Parameter grid for systematic evaluation
    data_param_grid = {
        "subject": ["fsaverage"],
        "nnz": [1, 5, 10],              # Number of active sources
        "orientation_type": ["fixed"],   # Source orientation
        "alpha_SNR": [0.0, 0.5, 0.9],  # Noise levels
    }
    
    print("Configuring benchmark experiment...")
    print(f"ERP amplitude: {ERP_config['amplitude']} nAm")
    print(f"Active sources: {data_param_grid['nnz']}")
    print(f"Noise levels: {data_param_grid['alpha_SNR']}")
    
    # Initialize benchmark
    benchmark = Benchmark(
        ERP_config=ERP_config,
        data_param_grid=data_param_grid,
        experiment_dir="./benchmark_results"
    )
    
    print("\nRunning benchmark experiment...")
    print("This would typically involve:")
    print("  1. Loading/generating leadfield matrices")
    print("  2. Simulating source activity for each condition")
    print("  3. Adding noise at specified SNR levels")
    print("  4. Applying source estimation algorithms")
    print("  5. Computing uncertainty estimates")
    print("  6. Evaluating calibration metrics")
    print("  7. Generating comprehensive reports")
    
    # Demonstrate benchmark configuration
    try:
        # This would run the full benchmark
        # results = benchmark.run(nruns=3)
        
        # Instead, show what the results would look like
        print("\nExpected results structure:")
        print("results.columns = [")
        print("    'subject', 'nnz', 'alpha_SNR', 'orientation_type',")
        print("    'localization_error', 'false_positive_rate',") 
        print("    'calibration_score', 'coverage_68', 'coverage_95',")
        print("    'sharpness', 'computation_time'")
        print("]")
        
        # Show example parameter combinations
        print(f"\nTotal parameter combinations: {len(data_param_grid['nnz']) * len(data_param_grid['alpha_SNR'])}")
        print("Parameter combinations:")
        for nnz in data_param_grid['nnz']:
            for alpha_snr in data_param_grid['alpha_SNR']:
                print(f"  - {nnz} active sources, α_SNR = {alpha_snr}")
        
    except Exception as e:
        print(f"Benchmark simulation: {str(e)}")
    
    print("\nBenchmark tutorial completed!")

#==============================================================================
# MAIN TUTORIAL FUNCTION
#==============================================================================

def run_complete_tutorial():
    """
    Run the complete CaliBrain library tutorial
    
    This function demonstrates the full workflow of the CaliBrain library,
    from leadfield generation to final evaluation and visualization.
    """
    print("=" * 80)
    print("CALIBRAIN LIBRARY TUTORIAL")
    print("Comprehensive Introduction to Brain Source Localization and Uncertainty Quantification")
    print("=" * 80)
    
    try:
        # 1. Build leadfield matrix
        leadfield_matrix = tutorial_leadfield_builder()
        
        # 2. Simulate source activity
        source_data, active_indices, time_vector = tutorial_source_simulator()
        
        # 3. Simulate sensor measurements
        sensor_data_clean, sensor_data_noisy = tutorial_sensor_simulator(
            leadfield_matrix, source_data
        )
        
        # 4. Estimate sources
        source_estimates = tutorial_source_estimator(
            sensor_data_clean, leadfield_matrix
        )
        
        # 5. Quantify uncertainty
        uncertainty_results = tutorial_uncertainty_estimator(
            sensor_data_clean, leadfield_matrix, source_estimates
        )
        
        # 6. Evaluate performance
        evaluation_metrics = tutorial_metric_evaluator(
            source_data, source_estimates, uncertainty_results, active_indices
        )
        
        # 7. Create visualizations
        tutorial_visualizer(
            source_data, sensor_data_clean, source_estimates, 
            active_indices, time_vector
        )
        
        # 8. Demonstrate benchmark workflow
        tutorial_benchmark()
        
        print("\n" + "=" * 80)
        print("TUTORIAL COMPLETED SUCCESSFULLY!")
        print("=" * 80)
        print("\nNext steps:")
        print("1. Explore the examples/ directory for more specific use cases")
        print("2. Read the API documentation for detailed parameter descriptions")
        print("3. Try the benchmark on your own data")
        print("4. Customize algorithms and parameters for your research needs")
        print("\nFor more information:")
        print("- Documentation: https://braindatalab.github.io/CaliBrain/")
        print("- GitHub: https://github.com/braindatalab/CaliBrain")
        print("- Issues: https://github.com/braindatalab/CaliBrain/issues")
        
    except Exception as e:
        print(f"\nTutorial error: {str(e)}")
        print("This is a demonstration - actual implementation may vary.")

if __name__ == "__main__":
    """
    Run the tutorial when script is executed directly
    """
    run_complete_tutorial()